# Preperation

In [1]:
import torch
import matplotlib.pyplot as plt
import yaml
import os
import sys
from copy import deepcopy
import h5py
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import numpy as np
import argparse


from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve

In [2]:
cd ..

/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN


In [3]:
cd src

/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/src


In [4]:
from model import CINN
from trainer import DNNTrainer, VAETrainer
import data_util
from documenter import Documenter
from plotter import Plotter
from matplotlib import cm
from myDataLoader import MyDataLoader
from calc_obs import *
from model import CINN, DNN
import plotting
import HighLevelFeatures as HLF
from XMLHandler import XMLHandler

In [5]:
cd ..

/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN


In [6]:
def load_trainer(directory, use_cuda=True):
    use_cuda = torch.cuda.is_available() and use_cuda
    device = 'cuda:0' if use_cuda else 'cpu' 


    with open(directory + "/params.yaml") as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
        
    doc = Documenter(params['run_name'], existing_run=True, basedir=directory,
                    log_name="log_jupyter.txt", read_only=True)
    
    particle = params.get("particle_type", "piplus")

    if "data_path" not in params:
        particle_type = params.get("particle_type", "pion")
        data_path = f"/remote/gpu06/ernst/Master_Thesis/Datasets/Dataset1/dataset_1_{particle_type}s_1.hdf5"
        params["data_path"] = data_path
        
    trainer = VAETrainer(params, device, doc)
    
    return trainer, params, device, doc

# Start calculations

In [7]:
directory = "/remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_03_21_095348_Default"
trainer, params, device, doc = load_trainer(directory)
trainer.load()


Using the directory: /remote/gpu06/ernst/Master_Thesis/vae_calo_challenge/CaloINN/results/2023_03_21_095348_Default
cuda:0


/remote/gpu06/ernst/envs/CaloINN/lib/python3.10/site-packages/torch/cuda/__init__.py:123: UserWarning: 
    Found GPU1 NVIDIA GeForce GTX TITAN which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))
/remote/gpu06/ernst/envs/CaloINN/lib/python3.10/site-packages/torch/cuda/__init__.py:123: UserWarning: 
    Found GPU2 NVIDIA GeForce GTX TITAN which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))


CVAE(
  (noise_layer_in): noise_layer()
  (encoder): Sequential(
    (fc0): Linear(in_features=541, out_features=4500, bias=True)
    (relu0): ReLU()
    (fc1): Linear(in_features=4500, out_features=1000, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=1000, out_features=150, bias=True)
    (relu2): ReLU()
    (fc_mu_logvar): Linear(in_features=150, out_features=100, bias=True)
  )
  (decoder): Sequential(
    (fc0): Linear(in_features=58, out_features=150, bias=True)
    (relu0): ReLU()
    (fc1): Linear(in_features=150, out_features=1000, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=1000, out_features=4500, bias=True)
    (relu2): ReLU()
    (fc_out): Linear(in_features=4500, out_features=533, bias=True)
  )
  (noise_layer_out): noise_layer()
  (norm_x_in): FixedLinearTransform()
  (norm_x_out): FixedLinearTransform()
)
